# Visualización Estadística en Python

Por **Eduardo Graells-Garrido (egraells@udd.cl)**.

En este notebook exploraremos distintos tipos de visualizaciones para estadística. Lo haremos con las bibliotecas `matplotlib` y `seaborn`, trabajando sobre `pandas`.

Trabajaremos con datos *reales*: la encuesta CASEN del año 2013 y la encuesta EOD del año 2012. La primera es de caracterización socio-económica del país y la segunda de transporte en Santiago. Específicamente, realizaremos un análisis exploratorio de estas encuestas, enfocándonos en preguntas específicas que plantearemos, y también a las preguntas que vayan surgiendo en clases.

Luego cruzaremos los datos de ambas encuestas para ver si encontramos algún link interesante entre la socio-economía y la movilidad de las comunas de Santiago.

Comencemos.

## Preámbulo

El primer paso es importar las bibliotecas necesarias para poder trabajar. Usaremos las bibliotecas `numpy` (vectores), `pandas`(DataFrames), `matplotlib` (visualización de bajo nivel) y `seaborn` (visualización estadística). 

Ya tienen experiencia con estas cuatro bibliotecas, pero hoy profundizaremos un poco más en las dos últimas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Esto configura la apariencia de los gráficos utilizando configuraciones de seaborn
sns.set(context='poster', style='ticks', palette='inferno', font='Linux Biolinum O')

# Esto es una instrucción de Jupyter que hace que los gráficos se desplieguen en el notebook
%matplotlib inline

## `matplotlib`

`matplotlib` es una biblioteca de **bajo nivel** para visualización en Python, con un paradigma **imperativo**.

  * De bajo nivel quiere decir que entrega las primitivas gráficas necesarias para crear visualizaciones más complejtas.
  * Imperativo quiere decir que se focaliza en las instrucciones que recibe la biblioteca, ya que no _abstrae_ las operaciones o codificaciones visuales de modo que operemos sobre éstas.

## `seaborn`

`seaborn` es una biblioteca que se construye _sobre_ `matplotlib`, y que incluye gráficos estadísticos y algunas funcionalidades avanzadas de modelamiento y reducción de complejidad. Esto permite _hacer más con menos código_. 

Al mismo tiempo, `seaborn` incluye opciones para trabajar con la codificación visual de manera más efectiva que `matplotlib`, por ejemplo, a través de la facilitación de elección de paletas de colores, o de la detección de parámetros siguiendo buenas prácticas.

Y para completar más esta oferta, `seaborn` es directamente compatible con `pandas`. `matplotlib` también, pero no de manera nativa, por lo que todo lo que se relaciona con `pandas` requiere más trabajo.

Oh, vemos que no solamente hizo el gráfico, sino que incluso etiquetó automáticamente el eje $x$.

Sin embargo, en `seaborn` la idea no es reemplazar los métodos de `matplotlib`. En realidad, lo que haremos será trabajar de manera diferente. Lo primero que debemos hacer es cambiar el formato de los datos. 

En `seaborn` se trabaja con **tidy data**. O datos ordenaditos.

En el dataframe `df` tenemos tres observaciones por fila. No es un dataframe ordenadito. Un dataframe ordenadito tendría una observación por fila. 

(Aquí hay dos explicaciones interesantes sobre _tidy data_: en [R](http://garrettgman.github.io/tidying/) y en [Python](http://www.jeannicholashould.com/tidy-data-in-python.html). El paper que explica el razonamiento detrás está [aquí](http://courses.had.co.nz.s3-website-us-east-1.amazonaws.com/12-rice-bdsi/slides/07-tidy-data.pdf)).

Afortunadamente, `pandas` nos permite convertir un dataframe en formato _longform_ a uno en formato _tidy_ con la función `pd.melt`.

## Analicemos Datos Reales

Exploremos y expandamos nuestra exploración de visualizaciones estadísticas a través del análisis de datos reales. Utilizaremos la Encuesta Origen-Destino de Viajes de Santiago (2012) y la encuesta CASEN (2013).

In [ ]:
def load_casen():
    # el archivo Casen2013.csv.gz contiene la base de datos en formato comprimido.
    casen = pd.read_csv('./input/2013_casen_survey/Casen2013.csv.gz', sep=',', 
                        encoding='iso-8859-1')
    
    # nos quedaremos solamente con la Región Metropolitana
    casen = casen[casen.region == 13].copy()
    
    # Esta es una manera fea de transformar los identificadores de comuna en nombres
    name_map = dict(zip([13101, 13102, 13103
    ,13104,13105,13106,13107,13108,13109,13110,13111,13112,13113,13114,13115,13116,13117,13118,13119,13120,13121,13122,13123
    ,13124,13125,13126,13127,13128,13129,13130,13131,13132,13201,13202,13203,13301,13302,13303,13401,13402,13403,13404,13501,
                     13502, 13503, 13504, 13505, 13601, 13602, 13603, 13604, 13605],
        map(lambda x: x.title(), ['Santiago', 'Cerrillos', 'Cerro Navia', 'Conchalí',
        'El Bosque', 'Estación Central', 'Huechuraba', 'Independencia', 'La Cisterna',
        'La Florida', 'La Granja', 'La Pintana',
        'La Reina','Las Condes','Lo Barnechea','Lo Espejo', 'Lo Prado',
        'Macul','Maipú','Ñuñoa','Pedro Aguirre Cerda','Peñalolén',
        'Providencia','Pudahuel','Quilicura','Quinta Normal','Recoleta','Renca',
        'San Joaquín','San Miguel','San Ramón','Vitacura','Puente Alto','Pirque',
        'San José de Maipo','Colina','Lampa','Tiltil','San Bernardo','Buin',
        'Calera de Tango','Paine','Melipilla','Alhué','Curacaví','María Pinto','San Pedro',
        'Talagante','El Monte','Isla de Maipo','Padre Hurtado','Peñaflor'
        ])))

    # aquí aplicamos la transformación y guardamos el resultado en una nueva columna
    casen['municipality'] = casen['comuna'].map(lambda x: name_map[x])
    return casen

casen = load_casen()
casen.shape

In [ ]:
# la encuesta tiene tantas columnas que hacemos esto para imprimirlas todas
pd.set_option('max_columns', 1000)

In [ ]:
casen.sample(3)

In [ ]:
print(list(casen.columns))

Como pueden ver, son muchísimas columnas -- es una encuesta compleja. En la carpeta `input` hay un archivo en PDF donde se explica el significado y posibles valores de cada respuesta.

### Relación entre Satisfacción con la Vida e Ingreso

Una de las preguntas que me llama mucho la atención es la que dice:

> Considerando todas las cosas, ¿qué tan satisfecho se encuentra con su vida?

La respuesta va de 1 (insatisfacción total) a 10 (satisfacción total).

Estudiaremos las relaciones entre esa pregunta, que tiene código `r20`, con otras variables del dataset.

Partamos con el ingreso del hogar, que tiene código `ytotcorh`. Podemos ver su distribución con un `sns.distplot`:

In [ ]:
sns.distplot(casen.ytotcorh)

¡Oh! Como los ingresos de la RM están sesgados, la distribución se ve bastante fea. Quizás deberíamos probar otro tipo de visualización, como las que entrega `sns.factorplot`:

In [ ]:
sns.factorplot(x='municipality', y='ytotcorh', data=casen, aspect=3, size=6, kind='bar')
plt.xticks(rotation=90);

Que bien, esto ya nos sirve para empezar a hacer análisis. Vemos que cada comuna tiene una barra asociada, que muestra el ingreso promedio por comuna, y una barra de error que representa la variabilidad, mostrando el intervalo de confianza del 95%. 

Lo que nos gustaría es poder ordenarlo. Para eso debemos calcular el orden nosotros:

In [ ]:
municipalities_by_income = (casen.groupby('municipality')
                            .aggregate({'ytotcorh': 'mean'})
                            .sort_values('ytotcorh'))

También aprovecharemos de usar un color fijo para las barras. No tiene sentido en este caso que las barras tengan colores diferentes:

In [ ]:
sns.factorplot(x='municipality', y='ytotcorh', data=casen, aspect=3, size=6, kind='bar',
               order=municipalities_by_income.index, color='grey')
plt.xticks(rotation=90);

¡Qué bien! Es un gráfico sencillo pero que a la vez comunica muchas cosas. Vemos la distribución del ingreso por comunas de manera impactante. Simplificando, esencialmente hay tres grupos de comunas: las muy, muy ricas, las ricas, y el resto, desde Santiago hacia abajo.

Digo _simplificando_ porque hasta aquí hemos obviado algunas cosas. Por un lado, cada hogar encuestado tiene una ponderación dentro de la muestra, llamada _factor de expansión_, que hemos ignorado. Y el ingreso que estamos viendo es a nivel de hogar, no de persona entrevistada. 

Y cada persona entrevistada es una observación en la base de datos. Es decir, hemos estado considerando duplicados en ese gráfico.

In [ ]:
# uso la conversión a int para que se muestre el valor completo.
casen.folio.astype(np.int).head(10)

Como se observa, hay folios repetidos.

In [ ]:
casen.ytotcorh.head(10)

Y por tanto, ingresos también. Hagamos un dataframe sin duplicados:

In [ ]:
casen_homes = casen.drop_duplicates('folio', keep='first').copy()
casen.shape, casen_homes.shape

In [ ]:
municipalities_by_home_income = (casen_homes.groupby('municipality')
                            .aggregate({'ytotcorh': 'mean'})
                            .sort_values('ytotcorh')
                            .rename(columns={'ytotcorh': 'home_income'}))
#municipalities_by_home_income

In [ ]:
sns.factorplot(x='municipality', y='ytotcorh', data=casen_homes, aspect=3, size=6, kind='bar',
               order=municipalities_by_home_income.index, color='grey')
plt.xticks(rotation=90);

Quizás al ver los gráficos nos cuesta encontrar la diferencia. Podemos hacer una rápida exploración para ver quiénes cambiaron.

In [ ]:
municipality_features = municipalities_by_income.join(municipalities_by_home_income)

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(municipalities_by_income.ytotcorh, municipality_features.home_income)
plt.plot([0, 4*10**6], [0, 4*10**6], ls="--", c=".3")
for idx, row in municipality_features.iterrows():
    plt.annotate(xy=(row.ytotcorh, row.home_income), s=idx)
sns.despine()

Las comunas que están más cerca de la diagonal son las que tenían menos error. Las comunas más ricas tenían un error más pronunciado, pero ya que ganaban más dinero era esperable.

Ahora empecemos a mirar la **satisfacción con la vida**.

In [ ]:
casen.r20.sample(10)

Lamentablemente no todas las personas respondieron esa pregunta, por eso hay `NaN`s. Veamos los resultados:

In [ ]:
pd.isnull(casen.r20).sum(), casen.shape[0]

Solamente 22 mil de 36 mil. Contemos la cantidad de respuestas utilizando el método `sns.countplot`, que es un histograma _categórico_. Recordemos que el histograma discretiza una variable continua en _bins_. En este caso eso no es completamente adecuado, porque no contaremos la cantidad de respuestas entre 4.5 y 5.5 (por dar un ejemplo). Queda así:

In [ ]:
sns.countplot(casen.r20, color='pink')
sns.despine()

Si vemos el PDF de la encuesta sabremos que el valor `99` es "No responde." No conozco la diferencia entre `99` y `NaN`. Seguro es semántica: quizás alguien en `NaN` no estaba habilitado para responder, mientras que `99` es _decidir_ no responder. Sin embargo, esto no se especifica en el documento.

Guardemos las respuestas válidas:

In [ ]:
casen_satisfaction = casen[casen.r20 <= 10].copy()
casen_satisfaction.shape

Y estimemos la distribución por comunas, manteniendo el orden que teníamos antes.

Esta vez usaremos `sns.factorplot` con `kind=point`, que es más fácil de comparar que el gráfico de barras, pero hay que considerar que el gráfico no parte en 0, por lo que las diferencias son más pequeñas de lo que puede observarse:

In [ ]:
sns.factorplot(x='municipality', y='r20', data=casen_satisfaction, aspect=3, size=6, kind='point',
               order=municipalities_by_home_income.index, color='grey', join=False)
plt.xticks(rotation=90);

Oh, pasa algo muy interesante. El orden de la pregunta `r20` no calza del todo con el orden de `ytotcorh`. Lo estudiaremos más adelante.

Antes, quizás podamos ver la distribución de `r20` de acuerdo a otras variables. Por ejemplo, el estado civil. En el PDF se menciona el siguiente diccionario:

  * 1: Casado
  * 2: Conviviente
  * 3: Anulado
  * 4: Separado
  * 5: Divorciado
  * 6: Viudo
  * 7: Soltero
  
El `sns.countplot` correspondiente es:

In [ ]:
sns.countplot(casen_satisfaction.ecivil, color='steelblue')
sns.despine()

Vemos que solamente las categorías 1, 2, 4, 6 y 7 tienen más de 1000 muestras. Enfoquémonos en ellas.

In [ ]:
g = sns.FacetGrid(data=casen_satisfaction, col='ecivil', row='sexo', hue='sexo',
                  aspect=1.4, size=5, col_order=[1,2,4,6,7], palette='inferno')
g.map(sns.countplot, 'r20', alpha=0.5)
g.add_legend()

Aunque es un gráfico completo, cuesta hacer las comparaciones. Recurramos nuevamente a `sns.factorplot`:

(pd: les dejo como misterio los valores de la variable `sexo`. ¿pueden adivinarla?)

In [ ]:
g = sns.factorplot(x='ecivil', y='r20', data=casen_satisfaction, order=[1,2,4,6,7], 
              kind='point', aspect=2, join=False, hue='sexo', dodge=True, palette='inferno')
g.set_xticklabels(labels=['Casado(a)', 'Conviviente', 'Separado(a)', 'Viudo(a)', 'Soltero(a)'])

Noten que usamos el parámetro `dodge=True` para poder diferenciar mejor los valores.

Los intervalos de confianza se interceptan en la mayoría de los casos, lo cual indica que posiblemente no hay diferencias significativas, a pesar de que los promedios sean diferentes.

Pasa algo interesante con la categoría de _les solteres_. ¡Es posible que la diferencia sea significativa! Pero recuerden: el gráfico no parte en base 0. La diferencia entre los promedios parece ser cercana a 0.25. 

Aparentemente, al estar solteres, el sexo `1` está _un pichintún_ más satisfecho con su vida que el sexo `2`.

Les dejo propuesta la corroboración utilizando algún test estadístico.

**Volvamos a las comunas**.

Estimemos la satisfacción con la vida promedio a nivel comunal, considerando los factores de expansión, y luego la comparamos entre municipalidades utilizando el puntaje estándar.

¿Por qué puntaje estándar?

Porque algo no cuadra. Estos valores, que parecen muy altos, no han sido interpretados correctamente. Se dice que [gran parte de los chilenos está satisfecho con su vida](http://www.plataformaurbana.cl/archive/2015/02/28/encuesta-casen-en-dos-anos-suben-de-63-a-70-los-chilenos-que-estan-felices-con-su-vida/). Sin embargo, Chile también es un país con [preocupantes índices de depresión](http://www.elmostrador.cl/agenda-pais/vida-en-linea/2017/02/23/chile-se-ubica-por-sobre-el-promedio-mundial-en-indice-de-depresion-segun-nuevo-informe-de-la-oms/) y de [estrés](https://www.publimetro.cl/cl/nacional/2016/10/27/80-chilenos-viven-alto-nivel-estres.html). Entonces, ¿cómo interpretar estos números?

¿Qué significa una satisfacción de `7`? Nadie lo sabe.

Proponemos que lo correcto es ver los índices de satisfacción de manera relativa. Lo que importa no es si alguien dice `7` de `10`, sino qué tan arriba o abajo del promedio alguien dice estar satisfecho.

Eso lo podemos medir con el puntaje estándar o _z-score_, definido así:

$$z = \frac{x - \mu}{\sigma}$$

El puntaje estándar de una observación es su diferencia con el promedio, medido en desviaciones estándar de la variable.

Sin embargo, antes de calcular $z$, necesitamos ver los factores de expansión:

In [ ]:
casen_satisfaction.loc[:,('r20', 'expr', 'expc', 'expr_r20')].sample(5)

Según el diccionario, esta pregunta debe usar el factor `expr_r20`. ¡Pero tiene muchos `NaN`!

In [ ]:
pd.isnull(casen_satisfaction.expr_r20).sum()

Nos quedamos con las observaciones no nulas en el factor:

In [ ]:
casen_satisfaction = casen_satisfaction[~pd.isnull(casen_satisfaction.expr_r20)].copy()
casen_satisfaction.shape

Y calculamos un `r20` ajustado:

In [ ]:
casen_satisfaction['adjusted_r20'] = casen_satisfaction.r20 * casen_satisfaction.expr_r20

Ahora agrupamos. Comento el código con lo que estamos haciendo:

In [ ]:
def z_score(col):
    return (col - col.mean()) / col.std()

# agrupamos por municipalidad
satisfaction = (casen_satisfaction.groupby('municipality')
                # calculamos la suma total de r20 y de r20 ajustado
                .aggregate({'r20': 'sum', 'adjusted_r20': 'sum'})
                # calculamos el factor expandido total para cada municipalidad
                .join(casen.groupby('municipality').aggregate({'expr_r20': 'sum'}))
                # esto nos permite calcular el promedio expandido
                .assign(exp_average=lambda x: x['adjusted_r20'] / x['expr_r20'])
                # calculamos z-score de este promedio
                .assign(z_satisfaction=lambda x: z_score(x['exp_average']))
                # ordenamos el dataframe resultante
                .sort_values('z_satisfaction'))
#satisfaction

Grafiquemos los resultados. Primero, el promedio:

In [ ]:
sns.factorplot(x='municipality', y='exp_average', data=satisfaction.reset_index(), 
               aspect=3, size=6, kind='bar',
               order=satisfaction.index, color='grey')
plt.xticks(rotation=90);

Noten que ya no hay barras de error. Esto es porque a `sns.factorplot` le hemos entregado una observación por comuna.

Veamos el puntaje estándar:

In [ ]:
sns.factorplot(x='municipality', y='z_satisfaction', data=satisfaction.reset_index(), 
               aspect=3, size=6, kind='bar',
               order=satisfaction.index, color='grey')
plt.xticks(rotation=90);

Wow, creo que se ve muy bien :)

Considerando cómo calculamos `r20` con expansión, hagamos lo mismo con el ingreso:

In [ ]:
# here we can use the municipal expansion (but it's not recommended... je je je)
casen_homes['adjusted_income'] = casen_homes.ytotcorh * casen.expc

income = (casen_homes.groupby('municipality')
                .aggregate({'ytotcorh': 'sum', 'adjusted_income': 'sum'})
                .join(casen_homes.groupby('municipality').aggregate({'expc': 'sum'}))
                .assign(exp_average=lambda x: (x['adjusted_income'] / x['expc']).astype(np.int))
                .rename(columns={'exp_average': 'average_income'}))
#income

Unamos ambos dataframes (ingreso y satisfacción) en uno solo, para poder comparar resultados:

In [ ]:
municipality_features = satisfaction.join(income).loc[:,('z_satisfaction', 'average_income')]

Veamos si están correlacionadas ambas características de las municipalidades. Podemos usar el método `sns.jointplot`. Calcularemos la correlación de _spearman_ puesto que nuestras variables no están distribuidas de manera normal (en tal caso hubiésemos podido utilizar la de _pearson_):

In [ ]:
from scipy.stats import spearmanr

sns.jointplot('average_income', 'z_satisfaction', 
              municipality_features,
              size=9, 
              stat_func=spearmanr)

No es una correlación significativa. Pero sí podemos ver que, a ingresos bajos y medios, está todo el espectro de satisfacción, pero que en ingresos altos, no hay comunas insatisfechas.

Este resultado no es nuevo. En un [estudio](http://science.sciencemag.org/content/312/5782/1908.full?casa_token=WDhcGsXTGqMAAAAA:UZXfIidKtiatn-6D5cZSxAIuQpcJ8JSUHX3Q9QSdKxJnZiu0Tk60s4uAT5u--vkzSEBpLMHP4gEABA) de Daniel Kahnemann, titulado _"Would You Be Happier If You Were Richer? A Focusing Illusion,"_ encontraron este mismo resultado. ¡Lo hubiésemos calculado antes! :D

## Encuesta Origen-Destino de Santiago, 2012

La Encuesta Origen-Destino de Santiago, efectuada por última vez el año 2012, es el instrumento principal que utilizan las autoridades para tomar decisiones respecto a transporte en la ciudad. Consistió en entrevistar a los residentes de más de 18000 hogares haciéndoles la siguiente pregunta:

> ¿Cuáles viajes hiciste ayer?

Las personas encuestadas responden a través de un diario de viaje. En este diario incluyen todos los datos pertinentes de sus viajes: a qué hora lo iniciaron, a qué hora terminaron, los puntos de origen y destino (coordenadas), el propósito del viaje, el/los modo(s) de viaje utilizados, etc. También incluye información socio-demográfica de cada persona que responde.

La encuesta es representativa a nivel comunal. Esto quiere decir que podemos sacar conclusiones sobre como se moviliza la población de Providencia, pero no de un barrio específico de la comuna. Puede ser que exista información de ese barrio específico, pero no podemos sacar conclusiones extrapolables al barrio completo.

Lo que haremos entonces es calcular algunas métricas a nivel comunal y correlacionarlas con lo que estimamos en la encuesta CASEN.

In [ ]:
# éste es un archivo preprocesado con datos de la encuesta
travel_survey = (pd.read_csv('./intermediate/stgo-travel-survey-2012.csv.gz')
                 .assign(departure_time=lambda x: pd.to_timedelta(x['HoraIni']))
                 .assign(arrival_time=lambda x: pd.to_timedelta(x['HoraFin']))
                 .assign(trip_duration=lambda x: (x['arrival_time'] - x['departure_time']) / pd.Timedelta(minutes=1))
                )
travel_survey.sample(5)

In [ ]:
travel_survey.columns

Limpiaremos un poco los viajes para que podamos entender mejor los resultados.

In [ ]:
# viajes con distancia mayor a 250 metros
travel_survey = travel_survey[(travel_survey.DistManhattan >= 250)
                  # el dataset no incluye el "especifique" así que lo sacamos
                  & (travel_survey.Proposito != 'Otra actividad (especifique)')
                  # estamos analizando la RM así que sacamos viajes fuera de ésta
                  & (~travel_survey.SectorOrigen.isin(['Exterior a RM'])) 
                  & (~travel_survey.Sector.isin(['Exterior a RM']))
                  & (~travel_survey.SectorDestino.isin(['Exterior a RM']))
                  # para los modos de viaje, tampoco se indica qué es "otros"
                  & ~(travel_survey.ModoDifusion == 'Otros')].copy()

In [ ]:
travel_survey.shape, travel_survey.columns

Visualicemos los propósitos de viaje de las personas:

In [ ]:
sns.countplot(y='Proposito', data=travel_survey, color='grey')
sns.despine()

Observamos que el propósito más común es "volver a casa." Esto es esperado, puesto que todos los demás propósitos implican un retorno a algún lugar, excepto casos especiales.

Ahora utilicemos `sns.distplot` para ver a qué hora comienzan los viajes en la RM:

In [ ]:
sns.distplot(travel_survey.HoraDeInicio)
plt.xlim([5, 24])
sns.despine()

Aquí hay dos cosas interesantes.

  1. Observen el patrón dentado del histograma. ¿A qué se debe? Recuerden que este dataset es una encuesta, por lo que la hora de inicio de viaje es la **reportada** por las personas. Tendemos a redondear los valores que reportamos. Nadie dice "salí a las una, doce minutos, 3 segundos," sino que se dice "salí a las una y cuarto." En ese aspecto, la estimación de la distribución utilizando Kernel Density Estimation funciona bien.
  2. En el gráfico se observan tres horas _peak_: las dos usuales (punta mañana y punta tarde) y un tercer peak a la hora de almuerzo. Ahora bien, dadas las rutinas diarias de las personas, seguramente cada peak obedece a propósitos de viaje distinto.
  
Entonces desagregemos por propósito de viaje:

In [ ]:
g = sns.FacetGrid(data=travel_survey, col='Proposito', col_wrap=3, aspect=2, sharey=False)
g.map(sns.distplot, 'HoraDeInicio')
g.add_legend()
g.set(xlim=[5,24])

Siguiendo la tónica que tuvimos cuando analizamos la CASEN, podríamos preguntarnos si existen diferencias de sexo en estos patrones. Esto lo podemos hacer de maneraq directa con el parámetro `hue='Sexo'`:

In [ ]:
g = sns.FacetGrid(data=travel_survey, col='Proposito', hue='Sexo',
                  col_wrap=3, aspect=2, sharey=False, palette='magma')
g.map(sns.distplot, 'HoraDeInicio')
g.add_legend()
g.set(xlim=[5,24])

Oh, vemos que algunos propósitos de viaje no exhiben diferencias en sus distribuciones, pero otros sí. Queda propuesta la interpretación de estos gráficos.

Ahora veamos la _partición modal_ de cada comuna. Partición modal se refiere a la distribución de los modos de transporte utilizados, a un nivel específico. Por ejemplo, si queremos saber la partición modal de los viajes al trabajo, podemos usar `sns.countplot`:

In [ ]:
sns.countplot(y='ModoDifusion', data=travel_survey[travel_survey.Proposito == 'Al trabajo'], 
              color='grey')
sns.despine()

Pero también podemos querer calcular la partición modal por sectores de la ciudad. Para eso, utilizaremos la operación `groupby` de `pandas` (nota: `FactorLaboralNormal` es el factor de expansión o ponderación para los viajes realizados en días laborales no estivales):

In [ ]:
travel_survey.groupby(['Sector', 'ModoDifusion']).aggregate({'FactorLaboralNormal': 'sum'})

Lo que quisiéramos hacer con esta tabla es normalizarla por sectores y visualizarla como un heatmap. Lo podemos hacer mezclando ambas cosas: la operación descrita y la función `sns.heatmap`:

In [ ]:
modal_partition = (travel_survey[travel_survey.Proposito == 'Al trabajo']
                   .groupby(['Sector', 'ModoDifusion'])
                   .aggregate({'FactorLaboralNormal': 'sum'})
                   .reset_index()
                   # Arriba vemos cómo queda el resultado.
                   # Para la función heatmap necesitamos que el formato sea "longform"
                   # Eso lo logramos con pivot_table
                   .pivot_table(index='Sector', values='FactorLaboralNormal', columns='ModoDifusion')
                   # Esto permite normalizar cada fila. 
                   # https://stackoverflow.com/questions/18594469/normalizing-a-pandas-dataframe-by-row#18594595
                   .pipe(lambda x: x.div(x.sum(axis=1), axis=0))
                  )

plt.figure(figsize=(9,6))
sns.heatmap(modal_partition, cmap='magma_r', annot=True, linewidth=1, square=True, robust=True)

Encontramos lo que todos sabemos: que el sector oriente es muy distinto al resto en lo que respecta al uso de tarjeta Bip! y autos. 

Pero también vemos que el centro también es diferente - quienes viven allí tienen una mayor tasa de viajes caminando al trabajo. ¡Qué envidia! 

Realicemos este mismo cálculo a nivel municipalidad:

In [ ]:
municipality_modal_partition = (travel_survey[travel_survey.Proposito == 'Al trabajo']
                   .groupby(['Comuna', 'ModoDifusion'])
                   .aggregate({'FactorLaboralNormal': 'sum'})
                   .reset_index()
                   .pivot_table(index='Comuna', values='FactorLaboralNormal', columns='ModoDifusion')
                   .fillna(0)
                   .pipe(lambda x: x.div(x.sum(axis=1), axis=0))
                  )

plt.figure(figsize=(16,16))
sns.heatmap(municipality_modal_partition, cmap='inferno_r', annot=True, linewidth=1, 
            robust=True)

Esto es más difícil de observar. Noten que las filas están ordenadas alfabéticamente, lo que permite su fácil localización, pero al mismo tiempo dificulta las comparaciones.

Quisiéramos que las filas y columnas que se parecen entre sí aparecieran juntas, de modo de poder hacer comparaciones de manera más fácil.

Para eso está `sns.clustermap`:

In [ ]:
g = sns.clustermap(municipality_modal_partition, cmap='viridis_r', annot=True, linewidth=1, figsize=(16,16),
            metric='euclidean', method='ward', square=False, robust=True, 
            yticklabels=True)

Ahora imaginen que con estos métodos se pueden visualizar otras variables. Queda propuesto ese ejercicio :)

Revisemos uno de los outpus más conocidos de la Encuesta Origen Destino: la matriz OD.

Definiremos una función que se llama `visualize_flow` y que recibe un dataframe de viajes.

In [ ]:
from sklearn.preprocessing import normalize

def visualize_flow(dataframe):
    flujos = (dataframe.groupby(['ComunaOrigen', 'ComunaDestino'])
              .aggregate({'FactorLaboralNormal': 'sum'})
              .reset_index())
    flujos_comunales = pd.pivot_table(flujos, index='ComunaOrigen', columns='ComunaDestino', values='FactorLaboralNormal').fillna(0)
    normalize(flujos_comunales, norm='l1', axis=1, copy=False)
    g = sns.clustermap(flujos_comunales, cmap='inferno_r', square=True, linewidths=1, 
                   metric='cosine', method='ward')
    # esto borra el clustermap
    plt.clf()
    plt.figure(figsize=(16, 16))
    sns.heatmap(g.data2d, cmap='inferno_r', square=True, linewidths=1, cbar_kws={'shrink': 0.4},
               xticklabels=True, yticklabels=True)

Lo que hace esta función es contar los viajes de una comuna a otra (de `ComunaOrigen` a `ComunaDestino`). Luego hace la misma normalización que hicimos en los heatmaps anteriores. 

Partamos viendo los viajes al trabajo/al estudio, luego los de salud y finalmente los de compras:

In [ ]:
visualize_flow(travel_survey[travel_survey['Proposito'].isin(['Al trabajo', 'Al estudio'])])

In [ ]:
visualize_flow(travel_survey[travel_survey['Proposito'].isin(['De salud'])])

In [ ]:
visualize_flow(travel_survey[travel_survey['Proposito'].isin(['De compras'])])

Nuevamente, el análisis queda propuesto.

¡Con esto ya tienen un buen set de visualizaciones estadísticas para explorar datos! :)

Pero todavía no terminamos.

### ¿Cómo cruzar ambos datasets?

Haremos un pequeño ejercicio. Digamos que queremos comparar variables entre la CASEN y la EOD.

La primera dificultad es que los nombres de comuna por hogar en la EOD tienen nombres que no son idénticos a los de la CASEN:

In [ ]:
travel_survey.Comuna.sample(5)

Además de que está todo en mayúscula, no incorpora los tildes.

Podemos hacer una función en Python utilizando el módulo `difflib`, que tiene una clase llamada `SequenceMatcher` que nos dice qué tanto se parece una secuencia a otra. 

Si encapsulamos `SequenceMatcher` en una función, y llamamos a esa función con Peñalolén como prueba, pasa esto:

In [ ]:
import difflib

def similar(a, b):
    return difflib.SequenceMatcher(None, a.lower(), b.lower()).ratio()

similar('PEÑALOLEN', 'Peñalolén')

El valor que aparece ahí va entre `0` (nada en común) y `1` (iguales). En este caso, vemos que funciona bastante bien.

Lo que haremos será crear un diccionario. Así:

In [ ]:
from itertools import product

muni_name_map = {}

for a, b in product(municipality_modal_partition.index, municipalities_by_home_income.index):
    if similar(a, b) > 0.75:
        print('Original:', a, '| Similar:', b, '. Ratio:', similar(a, b))
        muni_name_map[a] = b

Ahora crearemos una nueva columna utilizando el diccionario, y finalmente uniremos los dataframes con características municipales que hemos calculado para ambas encuestas:

In [ ]:
municipality_modal_partition['municipality'] = [muni_name_map[x] for x in municipality_modal_partition.index]
municipality_modal_partition.set_index('municipality', inplace=True)

In [ ]:
municipality_features = municipality_features.join(municipality_modal_partition)
municipality_features.head()

El siguiente paso es crear una matriz de correlaciones entre columnas de este dataframe:

In [ ]:
corr = municipality_features.corr(method='spearman')
corr.sample(5)

Una matriz que podemos visualizar con `sns.heatmap`:

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(corr, square=True, annot=True, robust=True, center=0, cmap='PuOr', linewidth=1,
           cbar=False)

Nuevamente, dejo propuesta la interpretación de este gráfico.

Guardaremos los resultados de nuestro análisis para explorarlo en la siguiente clase:

In [ ]:
municipality_features

In [ ]:
municipality_features.to_csv('./intermediate/municipality_features.csv')

In [ ]:
!head intermediate/municipality_features.csv